In [1]:
import numpy as np
import pandas as pd 

In [30]:
movies = pd.read_csv('/Users/AsadNaeem_1/Documents/UWindsor/Second Semester/COMP-8157 (Adv DB) LAB/Movie Recommender/tmdb Data/tmdb_5000_movies.csv')
credits = pd.read_csv('/Users/AsadNaeem_1/Documents/UWindsor/Second Semester/COMP-8157 (Adv DB) LAB/Movie Recommender/tmdb Data/tmdb_5000_credits.csv') 

In [31]:
movies = movies.merge(credits,on='title')

In [32]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [33]:
import ast

In [34]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 

In [35]:
movies.dropna(inplace=True)

In [36]:
movies['genres'] = movies['genres'].apply(convert)

In [37]:
movies['keywords'] = movies['keywords'].apply(convert)

In [38]:
import ast
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [39]:
movies['cast'] = movies['cast'].apply(convert)

In [40]:
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

In [41]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [42]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [43]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [44]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [45]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [46]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [47]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])

In [48]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))

In [49]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [50]:
vector = cv.fit_transform(new['tags']).toarray()

In [51]:
from sklearn.metrics.pairwise import cosine_similarity

In [53]:
similarity = cosine_similarity(vector)

In [54]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

In [55]:
recommend('Iron Man')

Iron Man 2
Iron Man 3
Avengers: Age of Ultron
Captain America: Civil War
The Avengers


In [56]:
import pickle

In [29]:
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))